# [HYPOTHESIS] Pending transfers that have intermediate errors are technical failures

### Context

We believe that there are specific intermediate/sender error codes that result in failed GP2GP transfers, and therefore should be interpreted as failed. We will know this to be true when we can see in the data that some intermediate/sender errors result in failures.

This notebook is to:
- identify if there are any error codes (sender, or intermediate) that we never recover from 
- for each error code, breakdown the transfers by the resulting outcome 
- present the probability of success per error code and rank in order of most likely to succeed

This analysis is for a 6 month time frame - From September 2020 to February 2021 (using transfers dataset).

In [1]:
import pandas as pd

In [2]:
transfer_file_location = "s3://<bucket-name>"
transfer_files = [
    "9-2020-transfers.parquet",
    "10-2020-transfers.parquet",
    "11-2020-transfers.parquet",
    "12-2020-transfers.parquet",
    "1-2021-transfers.parquet",
    "2-2021-transfers.parquet",
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]
transfers = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))

In [3]:
def group_by_errors(error_field, df):
    transfers_with_errors = df.groupby([error_field, 'status']).count()[['conversation_id']].unstack().fillna(0)
    transfers_with_errors.columns = transfers_with_errors.columns.get_level_values(1)
    transfers_with_errors["total_count"] = transfers_with_errors[["PENDING_WITH_ERROR","FAILED","INTEGRATED"]].sum(axis=1)
    return transfers_with_errors.sort_values(by=['INTEGRATED'], ascending=False)

In [4]:
def generate_percentages(df):
    transfers_with_percentage = df.iloc[:, 0:3].apply(lambda x: x / df.iloc[:, 3] * 100)
    transfers_with_percentage = transfers_with_percentage.sort_values(by=['INTEGRATED'], ascending=False)
    transfers_with_percentage = transfers_with_percentage.add_suffix(' (%)')
    return transfers_with_percentage.round(2)

## Transfers with sender errors

In [5]:
transfers_with_sender_errors = group_by_errors('sender_error_code', transfers)
transfers_with_sender_errors

status,PENDING_WITH_ERROR,INTEGRATED,FAILED,total_count
sender_error_code,,,,
20.0,2840.0,1111.0,1035.0,4986.0
19.0,311.0,76.0,44.0,431.0
14.0,10168.0,3.0,0.0,10171.0
23.0,418.0,3.0,2.0,423.0
6.0,897.0,0.0,0.0,897.0
7.0,723.0,0.0,0.0,723.0
10.0,3515.0,0.0,0.0,3515.0
21.0,2.0,0.0,0.0,2.0
24.0,134.0,0.0,1.0,135.0


In [6]:
transfers_with_sender_errors_percentage = generate_percentages(transfers_with_sender_errors)
transfers_with_sender_errors_percentage

status,PENDING_WITH_ERROR (%),INTEGRATED (%),FAILED (%)
sender_error_code,,,
20.0,56.96,22.28,20.76
19.0,72.16,17.63,10.21
23.0,98.82,0.71,0.47
14.0,99.97,0.03,0.00
6.0,100.00,0.00,0.00
7.0,100.00,0.00,0.00
10.0,100.00,0.00,0.00
21.0,100.00,0.00,0.00
24.0,99.26,0.00,0.74


## Conclusion

The table above shows that it is possible to recover from the following errors recorded by the sender and that transfers with these errors may be eventually integrated:
- error code **20** (22.28% integrated) - *Spine system  responded with an error*
- error code **19** (17.63% integrated) - *Sender check indicates that Requester is not the patient’s current healthcare provider*
- error code **23** (0.71% integrated) - *Message not sent because sending practice is not Large Message compliant*
- error code **14** (0.03% integrated) - *Message not sent because requestingpractice is not Large Message compliant*

## Transfers with intermediate errors

In [7]:
has_intermediate_errors = transfers["intermediate_error_codes"].apply(len) > 0
transfers_with_intermediate_errors_exploded = transfers[has_intermediate_errors].explode("intermediate_error_codes")
transfers_with_unique_intermediate_errors = transfers_with_intermediate_errors_exploded.drop_duplicates(subset=["conversation_id", "intermediate_error_codes"])

transfers_with_intermediate_errors = group_by_errors("intermediate_error_codes", transfers_with_unique_intermediate_errors)
transfers_with_intermediate_errors

status,FAILED,INTEGRATED,PENDING_WITH_ERROR,total_count
intermediate_error_codes,,,,
29,1207.0,397.0,42.0,1646.0
12,50.0,52.0,8.0,110.0
30,4.0,10.0,4.0,18.0
25,2.0,2.0,1.0,5.0
31,17.0,2.0,0.0,19.0
11,19.0,1.0,5.0,25.0
15,5.0,0.0,79.0,84.0
17,5.0,0.0,0.0,5.0
20,0.0,0.0,1.0,1.0


### Caveat

For transfers with multiple different error codes, each error code will be counted as a new transfer.

In [8]:
transfers_with_intermediate_errors_percentage = generate_percentages(transfers_with_intermediate_errors)
transfers_with_intermediate_errors_percentage

status,FAILED (%),INTEGRATED (%),PENDING_WITH_ERROR (%)
intermediate_error_codes,,,
30,22.22,55.56,22.22
12,45.45,47.27,7.27
25,40.00,40.00,20.00
29,73.33,24.12,2.55
31,89.47,10.53,0.00
11,76.00,4.00,20.00
15,5.95,0.00,94.05
17,100.00,0.00,0.00
20,0.00,0.00,100.00


## Conclusion

The table above shows that it is possible to recover from the following intermediate errors and that transfers with these errors may be eventually integrated:
- error code **30** (55.56% integrated) - *Large Message general failure*
- error code **12** (47.27% integrated) - *Duplicate EHR Extract received*
- error code **25** (40% integrated) - *Large messages rejected due to timeout duration reached of overall transfer*
- error code **29** (24.12% integrated) - *Large Message Re-assembly failure*
- error code **31** (10.53% integrated) - *The overall EHR Extract has been rejected because one or more attachments via Large Messageswere not received*
- error code **11** (4% integrated) - *Failed to successfully integrate EHR Extract*

## Merging sender and intermediate errors

In [9]:
transfers_with_sender_and_intermediate_errors = pd.concat([transfers_with_sender_errors, transfers_with_intermediate_errors])
transfers_with_sender_and_intermediate_errors.groupby(transfers_with_sender_and_intermediate_errors.index).sum()
transfers_with_sender_and_intermediate_errors.sort_values(by=['INTEGRATED'], ascending=False)

,PENDING_WITH_ERROR,INTEGRATED,FAILED,total_count
20.0,2840.0,1111.0,1035.0,4986.0
29.0,42.0,397.0,1207.0,1646.0
19.0,311.0,76.0,44.0,431.0
12.0,8.0,52.0,50.0,110.0
30.0,4.0,10.0,4.0,18.0
23.0,418.0,3.0,2.0,423.0
14.0,10168.0,3.0,0.0,10171.0
25.0,1.0,2.0,2.0,5.0
31.0,0.0,2.0,17.0,19.0
11.0,5.0,1.0,19.0,25.0


In [10]:
transfers_with_sender_and_intermediate_errors_percentage = generate_percentages(transfers_with_sender_and_intermediate_errors)
transfers_with_sender_and_intermediate_errors_percentage

,PENDING_WITH_ERROR (%),INTEGRATED (%),FAILED (%)
30.0,22.22,55.56,22.22
12.0,7.27,47.27,45.45
25.0,20.00,40.00,40.00
29.0,2.55,24.12,73.33
20.0,56.96,22.28,20.76
19.0,72.16,17.63,10.21
31.0,0.00,10.53,89.47
11.0,20.00,4.00,76.00
23.0,98.82,0.71,0.47
14.0,99.97,0.03,0.00


## Conclusion

The table above shows that it is possible to recover from the following errors and that transfers with these errors may be eventually integrated:
- error code **30** (55.56% integrated) - *Large Message general failure*
- error code **12** (47.27% integrated) - *Duplicate EHR Extract received*
- error code **25** (40% integrated) - *Large messages rejected due to timeout duration reached of overall transfer*
- error code **29** (24.12% integrated) - *Large Message Re-assembly failure*
- error code **20** (22.28% integrated) - *Spine system  responded with an error*
- error code **19** (17.63% integrated) - *Sender check indicates that Requester is not the patient’s current healthcare provider*
- error code **31** (10.53% integrated) - *The overall EHR Extract has been rejected because one or more attachments via Large Messageswere not received*
- error code **11** (4% integrated) - *Failed to successfully integrate EHR Extract*
- error code **23** (0.71% integrated) - *Message not sent because sending practice is not Large Message compliant*
- error code **14** (0.03% integrated) - *Message not sent because requestingpractice is not Large Message compliant*

None of the above errors overlapped between sender and intermediate errors, except error code 20 which was sometimes integrated when it was reported as a sender error (1111/4986), and was not integrated when it was reported as an intermediate error (0/1).